In [ ]:
%cd ..
%load_ext autoreload
%autoreload 2


In [ ]:
from sejong_music.era_inference import EraTransformer

era_transformer = EraTransformer('wandb/run-20240405_155230-52nfcuf5/files/checkpoints')
gen_str = open('music_score/chwipunghyeong_gen.txt').read()
notes_set, score = era_transformer.infer_on_gen_str(gen_str, 'piri', 30, onset_ratio=1.2)
score.show()

In [ ]:
from sejong_music.jg_code import JeongganPiece

inst = 'piri'
idx = 0
self = era_transformer

piece = JeongganPiece(None, gen_str=gen_str, inst_list=[inst])
x = piece.convert_tokens_to_roll(piece.sliced_parts_by_measure[idx][inst], inst)
x = self.convert_to_input_format(x)

In [12]:
import torch
from sejong_music.jg_to_staff_converter import JGToStaffConverter
from sejong_music.era_align_utils import EraAlignPairSet

tokens = piece.sliced_parts_by_measure[idx][inst]
notes = JGToStaffConverter.convert_to_notes(tokens)
JGToStaffConverter.get_duration_of_notes(notes)
JGToStaffConverter().create_m21_notes(notes)

features = []
for note in notes:
  note_feature = ['era1', f'pitch{note.midi_pitch}', EraAlignPairSet.convert_duration(note.duration/2), 'strong']
  features.append(note_feature)

masker_input = torch.LongTensor(era_transformer.masker.tokenizer(features))
era_transformer.masker.align_model(masker_input.unsqueeze(0)).squeeze()

tensor([0.7171, 0.6460, 0.5830, 0.5189, 0.4746, 0.1499, 0.0526, 0.3584, 0.3573,
        0.1712, 0.2594, 0.2960, 0.1404, 0.2429, 0.2827],
       grad_fn=<SqueezeBackward0>)

In [ ]:
era_transformer.masker.tokenizer.vocab

In [ ]:
new_vocab = []
for word in era_transformer.masker.tokenizer.vocab:
  if 'pitch' not in word:
    new_vocab.append(word)
  elif word == 'pitch0':
    new_vocab.append(word)
  else:
    new_vocab.append(f'pitch{int(word[5:])+12}')
new_vocab

In [ ]:
idx=0
inst = 'piri'
self = era_transformer

piece = JeongganPiece(None, gen_str=gen_str, inst_list=[inst])
x = piece.convert_tokens_to_roll(piece.sliced_parts_by_measure[idx][inst], inst)
x = self.convert_to_input_format(x)
x, loss_mask = self.masker.mask_except_note_onset(x)
x, loss_mask = self.aug_form_to_input_form(x), self.aug_form_to_input_form(loss_mask)
x = self.unmask_pitch_and_ornaments(x.to(self.device), loss_mask.to(self.device), 3)


In [ ]:
roll = self.tokenizer.decode(x)
roll

In [ ]:
import argparse
import datetime
import copy
from pathlib import Path
from omegaconf import OmegaConf
from fractions import Fraction
from tqdm.auto import tqdm

import torch
from music21 import converter, stream, note as m21_note 
import numpy as np

from sejong_music import model_zoo, jg_code, inference
from sejong_music.yeominrak_processing import OrchestraScoreSeq, ShiftedAlignedScore, Tokenizer
from sejong_music.model_zoo import JeongganTransSeq2seq, JeongganBERT
from sejong_music.decode import MidiDecoder, OrchestraDecoder
from sejong_music.inference_utils import prepare_input_for_next_part, get_measure_specific_output, fix_measure_idx, fill_in_source, get_measure_shifted_output, recover_beat, round_number
from sejong_music.inference import JGInferencer
from sejong_music.jg_to_staff_converter import JGToStaffConverter
from sejong_music.jg_code import JeongganDataset, JeongganTokenizer, JGMaskedDataset, JeongganPiece
from sejong_music.full_inference import Generator

In [ ]:
state_dir = Path('wandb/run-20240405_155230-52nfcuf5/files/checkpoints')
config_fn = state_dir / 'config.yaml'
json_fn = state_dir / 'tokenizer_vocab.json'
ckpt_fn = state_dir / 'inst_0' / 'iter46400_model.pt'

assert config_fn.exists() and json_fn.exists() and ckpt_fn.exists()

config = OmegaConf.load(config_fn)
tokenizer = JeongganTokenizer(None, None, True, json_fn=json_fn)
model = JeongganBERT(tokenizer=tokenizer, config=config.model)
model.load_state_dict(torch.load(ckpt_fn, map_location='cpu'))
dataset = JGMaskedDataset(is_valid=True, tokenizer=tokenizer, augment_param=config.aug)
augmentor = dataset.augmentor

In [ ]:
def convert_to_input_format(roll_array:np.ndarray):
  assert roll_array.ndim == 2
  roll_list = roll_array.tolist()
  roll_list = [ ['start'] * len(roll_list[0])] + roll_list + [ ['end'] * len(roll_list[0])]
  tokens = tokenizer(roll_list)
  return torch.LongTensor(tokens)

def aug_form_to_input_form(x:torch.Tensor):
  assert x.ndim == 3
  return x.permute(2,0,1).flatten(0,1)

x = dataset.entire_segments[0]['daegeum']
x = convert_to_input_format(x)
x, loss_mask = augmentor.mask_except_start_beat(x)
x, loss_mask = aug_form_to_input_form(x), aug_form_to_input_form(loss_mask)

In [ ]:
gen_str = open('music_score/chwipunghyeong_gen.txt').read()
piece = JeongganPiece(None, gen_str=gen_str, inst_list=['piri'])

x = piece.convert_tokens_to_roll(piece.sliced_parts_by_measure[30]['piri'], 'piri')
x = convert_to_input_format(x)
# x, loss_mask = augmentor.mask_except_start_beat(x)
x, loss_mask = augmentor.mask_except_note_onset(x)
x, loss_mask = aug_form_to_input_form(x), aug_form_to_input_form(loss_mask)

x = bert_inference(model, x, loss_mask, 3)

In [ ]:
import matplotlib.pyplot as plt
from  torch.nn.functional import one_hot

plt.imshow(one_hot(x[:, 0]).T, aspect='auto', interpolation='nearest')

In [ ]:
def bert_inference(model, x, loss_mask, n):
  with torch.inference_mode():
    pitch_mask_remaining = loss_mask[:,0].sum()
    while pitch_mask_remaining >0:
      pred, _ = model(x.unsqueeze(0))
      pred = pred[0] # (T, C)
      pitch_pred = pred[:, 0]
      pitch_mask = loss_mask[:, 0]

      is_masked = pitch_mask==1
      pitch_masked_idx = torch.where(is_masked)[0]
      
      masked_pitch_prob = pitch_pred[is_masked].softmax(dim=-1)
      high_prob_idx = masked_pitch_prob.max(dim=-1).values.argsort(descending=True)[:n]
      high_prob_pitches = masked_pitch_prob[high_prob_idx].argmax(dim=-1)
      
      selected_idx = pitch_masked_idx[high_prob_idx]
      x[selected_idx, 0] = high_prob_pitches 
      loss_mask[selected_idx, 0] = 0
      pitch_mask_remaining = loss_mask[:,0].sum()
    
    orn_mask_remaining = loss_mask[:,1].sum()  
    while orn_mask_remaining >0:
      pred, _ = model(x.unsqueeze(0))
      pred = pred[0] # (T, C)
      orn_pred = pred[:, 1]
      orn_mask = loss_mask[:, 1]

      is_masked = orn_mask==1
      orn_masked_idx = torch.where(is_masked)[0]
      
      masked_orn_prob = orn_pred[is_masked].softmax(dim=-1)
      high_prob_idx = masked_orn_prob.max(dim=-1).values.argsort(descending=True)[:n]
      high_prob_orns = masked_orn_prob[high_prob_idx].argmax(dim=-1)
      
      selected_idx = orn_masked_idx[high_prob_idx]
      x[selected_idx, 1] = high_prob_orns 
      loss_mask[selected_idx, 1] = 0
      orn_mask_remaining = loss_mask[:,1].sum()
  return x

In [ ]:
n = 3
pitch_mask_remaining = loss_mask[:,0].sum()
while pitch_mask_remaining >0:
  pred, _ = model(x.unsqueeze(0))
  pred = pred[0] # (T, C)
  pitch_pred = pred[:, 0]
  pitch_mask = loss_mask[:, 0]

  is_masked = pitch_mask==1
  pitch_masked_idx = torch.where(is_masked)[0]
  
  masked_pitch_prob = pitch_pred[is_masked].softmax(dim=-1).detach()
  high_prob_idx = masked_pitch_prob.max(dim=-1).values.argsort(descending=True)[:n]
  high_prob_pitches = masked_pitch_prob[high_prob_idx].argmax(dim=-1)
  
  selected_idx = pitch_masked_idx[high_prob_idx]
  x[selected_idx, 0] = high_prob_pitches 
  loss_mask[selected_idx, 0] = 0
  pitch_mask_remaining = loss_mask[:,0].sum()

orn_mask_remaining = loss_mask[:,1].sum()  
while orn_mask_remaining >0:
  pred, _ = model(x.unsqueeze(0))
  pred = pred[0] # (T, C)
  orn_pred = pred[:, 1]
  orn_mask = loss_mask[:, 1]

  is_masked = orn_mask==1
  orn_masked_idx = torch.where(is_masked)[0]
  
  masked_orn_prob = orn_pred[is_masked].softmax(dim=-1).detach()
  high_prob_idx = masked_orn_prob.max(dim=-1).values.argsort(descending=True)[:n]
  high_prob_orns = masked_orn_prob[high_prob_idx].argmax(dim=-1)
  
  selected_idx = orn_masked_idx[high_prob_idx]
  x[selected_idx, 1] = high_prob_orns 
  loss_mask[selected_idx, 1] = 0
  orn_mask_remaining = loss_mask[:,1].sum()

In [ ]:
import music21
import copy
from typing import List
from sejong_music.jeonggan_utils import JGConverter, GencodeConverter, RollToJGConverter
from sejong_music.jg_to_staff_converter import JGToStaffConverter


roll = tokenizer.decode(x)

converter = RollToJGConverter()
jg_omr_str = converter(roll)
gen_str = GencodeConverter.convert_lines_to_gencode(jg_omr_str.split('\n'))
staff_converter = JGToStaffConverter()
mnotes, score = staff_converter(gen_str)

org_token = piece.sliced_parts_by_measure[30]['piri']
org_nots, org_score = staff_converter(org_token)

total_score = music21.stream.Score()
total_score.insert(0, score)
total_score.insert(0, org_score)
total_score.show()

In [ ]:
roll

In [ ]:
mnotes

In [ ]:
jg_omr_str

In [ ]:
roll

In [ ]:
from sejong_music.jg_code import JeongganPiece

JeongganPiece('music_score/chwipunghyeong.txt')

In [ ]:
with open('music_score/chwipunghyeong.txt') as f:
  jg_omr_str = f.read()
# jg_omr_str.split('\n')
gen_str =GencodeConverter.convert_lines_to_gencode(jg_omr_str.split('\n'))
gen_str

In [ ]:
pitch_mapping = {'배대':'배황', '배협':'배태', '배유':'배중', '배이':'배임', '배무':'배남',
'대':'황', '협':'태', '유':'중', '이':'임', '무':'남', '청대':'청황', '청협':'청태'}

outputs = []
for token in gen_str.split(' '):
  if token in pitch_mapping:
    outputs.append(pitch_mapping[token])
  else:
    outputs.append(token)
outputs = ' '.join(outputs)

with open('music_score/chwipunghyeong_gen.txt', 'w') as f:
  f.write(outputs)

In [ ]:
score.show()

In [ ]:
org_token = dataset.all_pieces[0].sliced_parts_by_measure[0]['daegeum']
org_notes, org_score = staff_converter(org_token)

In [ ]:
import music21
total_score = music21.stream.Score()

total_score.insert(0, score)
total_score.insert(0, org_score)
total_score.show()

In [ ]:
masked_pitch_prob.max(dim=-1).values.argsort(descending=True)

In [ ]:
plt.imshow(masked_pitch_prob.detach().cpu().numpy().T, aspect='auto')

In [ ]:
pred.squeeze().shape, (loss_mask==1).shape

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
loss_mask.shape

In [ ]:
pred.squeeze()[loss_mask==1].shape

In [ ]:
masked_token_prob = pred.squeeze()[loss_mask==1].softmax(dim=-1).detach()
masked_token_prob.shape

In [ ]:
plt.imshow(pred.squeeze()[loss_mask==1].softmax(dim=-1).detach().T)